In [3]:
from rdkit import Chem
from rdkit.Chem import QED
import pandas as pd
import time
from rdkit.Chem import RDConfig
import os
import sys
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
# now you can import sascore!
import sascorer
from rdkit.Chem import MACCSkeys
import joblib
import numpy as np
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import Draw

# 训练集

In [4]:
ftarget = '../data/train/EP2.csv'
df_target = pd.read_csv(ftarget)

fadmet = '../data/train/EP4.csv'
df_admet = pd.read_csv(fadmet)


df = pd.concat([df_target, df_admet])
train_data_list = df['smiles'].values

# 读取Log

In [5]:
fname = '../log/eval_2000245_24_EP2nEP4.log'
# fname = '../log/eval_gtp2_2421_EP2nEP4_200.log'
# fname = '../log/eval_gtp2_2421_EP4_150.log'
# fname = '../log/eval_2000245_4_EP2nEP4.log'
# fname = '../log/eval_2000245_9_EP2nEP4.log'
f =  open(fname,'r')
lines = f.readlines()
print(len(lines))

predicts = []
truth = []
for line in lines:
    if 'startofsmiles' in line:
        # tmp = line.split('>')[1].strip()
        tmp = line.split('<|startofsmiles|>')
        # predicts.append(tmp[1].split(' ')[0].strip())
        predicts.append(tmp[-1].strip())

        # break
    elif 'Reference' in line:
        tmp = line.split('Reference smiles: ')[1].strip()
        truth.append(tmp)
    else:
        pass

nums = len(truth)

58284


# 筛选

In [6]:
clf_EP4 = joblib.load("SVC_EP4.m")
clf_EP2 = joblib.load("SVC_EP2.m")

outputs = []

for i in range(nums):
    smile = predicts[i]
    if smile in train_data_list:
        continue
    try:
        mol=Chem.MolFromSmiles(smile)
    except Exception as e:
        # print(smile)
        # break
        continue

    if mol is None:
        print(smile)
        continue

    fp = MACCSkeys.GenMACCSKeys(mol)
    fp_bits = fp.ToList()
    label_EP4 = clf_EP4.predict(np.array(fp_bits).reshape(1,-1)).item()
    label_EPHX2 = clf_EP2.predict(np.array(fp_bits).reshape(1,-1)).item()

    outputs.append((smile, label_EP4, label_EPHX2))

[15:57:41] SMILES Parse Error: unclosed ring for input: 'Cc1ccccc1OCC1C(=O)OCC1C(O)CCOC1C'
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 11 12 14


Cc1ccccc1OCC1C(=O)OCC1C(O)CCOC1C
CCCC(=NCC1CCCO1)c1c(C)c(C#N)c(=O)n1C1CCNCC1


[15:57:42] Can't kekulize mol.  Unkekulized atoms: 1 2 5 6 15 16 17 18 19
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 1 2 15 16 17 18 19 26 27
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 18 19 20 21 22
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 11 12 13 14
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 10 11 13 14 15 16 22 23 24
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 17 18 19
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 17 18 19
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 13 14 29
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 13 14 29
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 13 14 29
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 19 20 37


Cc1c(C=O)c2c(OCc3ccccc3)nnc2cc1N1CCC(N)C1
Cc1c(CCc2ccc(C(F)(F)F)cc2)nnc2cc(N3CCC(N)C3)cnn12
Cc1cc2c(NCCc3ccc(S(N)(=O)=O)cc3)nnc2cc1C1CNNC1
Cc1nc2cc(C3CNNC3)ccc2c1OCc1ccccc1
CN1CCN(CCCN=Cc2c(O)cc3cc(C4CCOC4)ccc23)CC1
O=C(NS(=O)(=O)c1cnccc1-n1cnn(Cc2ccn(-c3ccccc3)c2=O)c1=O)N1CCCC1
O=C(CS(=O)(=O)c1cnccc1-n1ccn(Cc2ccn(-c3ccccc3)c2=O)c1=O)N1CCCC1
O=C(NS(=O)(=O)c1cnccc1-n1ccn(Cc2ccn(-c3ccccc3)c2=O)c1)N1CCCC1
O=C(NS(=O)(=O)c1ccccc1-n1ccn(Cn2ccn(-c3ccccc3)c2=O)n1)N1CCCC1
O=C(CS(=O)(=O)c1ccccc1-n1ccn(Cn2ccn(-c3ccccc3)c2=O)n1)N1CCCC1
Cc1oncc1C(=O)NS(=O)(=O)c1cnccc1-n1cnn(C(C)c2ccc(-c3ccccc3)c(=O)o2)c1


[15:57:42] Can't kekulize mol.  Unkekulized atoms: 2 3 5 6 7 8 9 16 18
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 1 2 4 11 13 14 15 24 26
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 14 15 16 17 18
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 17 18 19 20 21 22 23 24 25 26 27 28
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25 26 27 28
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 16 18 19 20 21 22 23 24 25 26 27 28
[15:57:42] Can't kekulize mol.  Unkekulized atoms: 15 16 18 19 20 21 22


...

2c(F)cccc2N1CCN1CCOCCOCC1COc1ccccc1OCC1CNCCN1CC=Cc1ccccn1
2CCN(CC1COCCN1CCOCC1COc1ccccc1C=CCN1CCOCC1)C(=NC1CCOCC1)NN1CCOCC1
11=C1CCC(C=CCNc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]c1=C1CCN(C=NCCN2CCOCC2)C1
1=C(NCCN2CCCCC2)N(CCOCCN2CCCCC2)C(=Cc2cccn2CNCc2cnc[nH]2)COC1=NCCN1CCCCC1
1<|>1=C1CCC(C=CCCc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]1
1=C1CSC(C=CCCc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]1
1=c1ccn(C=CCCN2CCCCC2)c1CCOCCN1CCc2ccccc2[nH]c1=C1CCN(C=NCCN2CCOCC2)C1
1=C(NCCCCN2CCOCC2)N(CCCCN2CCOCC2)C(=Cc2cccn2C)C1
1=C1CCC(C=CCCc2ccccc2)N1CCOCCN1CCN(C)CC1CNC(C)=C1CCN(C=CNCc2ccccc2)C1CNCc1cnc[nH]1
1=c1ccc(C=CCCN2CCCCC2)n1CCOCCN1CCOCC1C=C1CCC(C=CCCN2CCOCC2)N1CCCOc1cnc[nH]1
1=C1NCC(C=CCNc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]c1=C1CCN(C=NCCN2CCOCC2)C1
1=c1occ(C=CCCN2CCCCC2)c1OCC(C=CCNc1ccccn1)N1CCCOc2ccccc2C1
COc1ccc(C=CC(=O)NS(=O)(=O)c2ccc()s2)c(C=CCN2CCOCC2)c1
COc1ccc(C=CC(=O)NS(=O)(=O)c2ccc()s2)c(C=CCN2CCOCC2)c1
2*****1<|>LOW<|>LOW<|>N1CCN(CC=Cc2ccccc2COC2CNCCN2CCN2CCOCC2COc2ccccc2CNC2CCC2)CC1
C(=Cc1ccccc1)CN1CCOCC1CCN1CCOCCN

[15:57:47] SMILES Parse Error: syntax error while parsing: 11=C1CCC(C=CCNc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]c1=C1CCN(C=NCCN2CCOCC2)C1
[15:57:47] SMILES Parse Error: Failed parsing SMILES '11=C1CCC(C=CCNc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]c1=C1CCN(C=NCCN2CCOCC2)C1' for input: '11=C1CCC(C=CCNc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]c1=C1CCN(C=NCCN2CCOCC2)C1'
[15:57:47] SMILES Parse Error: syntax error while parsing: 1=C(NCCN2CCCCC2)N(CCOCCN2CCCCC2)C(=Cc2cccn2CNCc2cnc[nH]2)COC1=NCCN1CCCCC1
[15:57:47] SMILES Parse Error: Failed parsing SMILES '1=C(NCCN2CCCCC2)N(CCOCCN2CCCCC2)C(=Cc2cccn2CNCc2cnc[nH]2)COC1=NCCN1CCCCC1' for input: '1=C(NCCN2CCCCC2)N(CCOCCN2CCCCC2)C(=Cc2cccn2CNCc2cnc[nH]2)COC1=NCCN1CCCCC1'
[15:57:47] SMILES Parse Error: syntax error while parsing: 1<|>1=C1CCC(C=CCCc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]1
[15:57:47] SMILES Parse Error: Failed parsing SMILES '1<|>1=C1CCC(C=CCCc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]1' for input: '1<|>1=C1CCC(C=CCCc2ccccc2)N1CCOCCN1CCc2ccccc2[nH]1'
[15:57:47] SMILES Parse Erro

c1ccc(CN2CCOc3c2cc(-c2cnn4cccnc34)cc3c2OCCN3)nc1


[15:57:47] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13 14 15 17 18 19 20 21 22 23 24


KeyboardInterrupt: 

In [18]:
succeeded = []
for item in outputs:
    if item[1] == item[2] == 1:
        succeeded.append(item)
    # else:
    #     succeeded.append(item)

In [19]:
valid_num = len(outputs)
succeeded_number = len(succeeded)
sr = succeeded_number / valid_num
print(f"共生成{valid_num}个分子，成功{succeeded_number}个，成功率{sr}.")

共生成14336个分子，成功812个，成功率0.056640625.


In [20]:
iso_succeeded = list(set(succeeded))

In [21]:
len(iso_succeeded)

503

In [22]:
filtered = []
for i in range(len((iso_succeeded))):
    smiles = iso_succeeded[i][0]
    mol = Chem.MolFromSmiles(smiles)

    qed=QED.qed(mol)
    sas = sascorer.calculateScore(mol)

    # print(f"Qed: {qed}, SAs: {sas}")
    if qed < 0.6 or sas > 4:
        continue

    # if qed < 0.7 or sas > 3:
    #     continue

    save_name = f'EP2nEP4_{i}.png'
    save_path = f'../fig/EP2nEP4_{i}.png'

    # 画分子方法1
    # d = rdMolDraw2D.MolDraw2DSVG(500, 500) # or MolDraw2DCairo to get PNGs
    # rdMolDraw2D.PrepareAndDrawMolecule(d, mol)
    # d.FinishDrawing()
    # svg = d.GetDrawingText()
    # # save_name = f'../fig/EP2nEP4_{i}.png'
    # with open(save_path, 'w') as f:
    #     f.write(svg)

    # 画分子方法2
    # d = rdMolDraw2D.MolDraw2DSVG(500, 500) # or MolDraw2DCairo to get PNGs
    d = rdMolDraw2D.MolDraw2DCairo(500, 500) # or MolDraw2DCairo to get PNGs
    rdMolDraw2D.PrepareAndDrawMolecule(d, mol)
    d.FinishDrawing()
    d.WriteDrawingText(save_path)   


    # 画分子方法2
    # Draw.MolToFile(mol, save_name)

    filtered.append([i,smiles,qed,sas,save_name])

In [23]:
len(filtered)

444

In [24]:
from openpyxl import Workbook
from openpyxl.drawing.image import Image
# from PIL import Image
wb = Workbook()
ws = wb.active

In [25]:
ws['A1'] = 'ID'
ws['B1'] = 'smiles'
ws['C1'] = 'QED'
ws['D1'] = 'SAs'
ws['E1'] = 'image'

for i in range(len(filtered)):
    item = filtered[i]
    ID = item[0]
    smiles = item[1]
    qed = item[2]
    sas = item[3]
    file_name = item[4]

    ws[f'A{i+2}'] = ID
    ws[f'B{i+2}'] = smiles
    ws[f'C{i+2}'] = qed
    ws[f'D{i+2}'] = sas
    ws[f'E{i+2}'] = f'<table><img src="D:\\shared_workspace\\nlbmg\\fig\\{file_name}" width="300" height="300"></table>'

    # img = Image(file_path)
    # img = Image.open(file_path)
    # ws.add_image(img, f'E{i+2}')

# <table><img src="C:\MyPicture\2020-03-24_205819.png" width="120" height="148">

In [26]:
wb.save('./EP2nEP4.xlsx')

# 验证BBB

In [2]:
from rdkit.Chem import MACCSkeys
def maccs_from_mol(mol):
    maccs = [1]
    fp = MACCSkeys.GenMACCSKeys(mol)
    fp_bits = list(fp.GetOnBits())
    for fp in fp_bits:
        maccs.append(f'{fp}:1')
    return maccs

In [3]:
smiles_list = [
    'CCN(CC)Cc1ccc(CNC(=O)Nc2ccc(Cl)c(Cl)c2)s1',
    'CCCCCCNC(=O)NC1CCN(c2ccc(F)cc2)C1',
    'CC(=O)NCC1CN(c2ccc(-c3cccc(OC(F)(F)F)c3)c(F)c2)C(=O)O1',
    'CCCCC(NC(=O)Nc1cccc(C(F)(F)F)c1)C(=O)O',
    'O=C(O)c1ccc(CCCN2C(=O)CCC2C=CCN2CCC2)s1',
    'O=C(O)c1ccc(CCCN2C(=O)CCC2C=CC(O)CCc2ccco2)o1',
    'O=C(O)c1ccc(CCCN2C(=O)CCC2C=CC(O)C2CCO2)s1',
    'O=C1CCC(C=CC(O)c2cccc(-c3cccc(CO)c3)c2)N1',
    'O=C(O)c1ccc(CCCN2C(=O)CCC2C=CCN2CC(F)C2)s1',
    'CCNC(=O)C(C)NC(=O)C1CC(=O)N(CCc2ccc(OC)cc2)C1',
    'COC(=O)CC1CC(COCc2cccc(-c3ccccc3)c2)NC1=O',
    'O=C(CSC1CCNC1=O)NCCOCC(F)(F)F'
]

In [4]:
outputs = []
for smiles in smiles_list:
    m=Chem.MolFromSmiles(smiles)
    maccs = maccs_from_mol(m)
    outputs.append(maccs)

In [5]:
outputs = pd.DataFrame(data=outputs)
outputs.to_csv(f'./maccs_EP2nEP4.csv', index=False, header=False, sep=' ')

In [ ]:
bbb_list = [
    0.933858,
    0.941349,
    0.977456,
    0.922584,
    0.839427,
    0.881569,
    0.560391,
    0.97451,
    0.851953,
    0.271118,
    0.832895,
    0.965616
]

In [21]:
smiles_dict = {
    'refined_3': ['CCN(CC)Cc1ccc(CNC(=O)Nc2ccc(Cl)c(Cl)c2)s1',0.933858],
    'new_49':['CCCCCCNC(=O)NC1CCN(c2ccc(F)cc2)C1',0.941349],
    'new_58':['CC(=O)NCC1CN(c2ccc(-c3cccc(OC(F)(F)F)c3)c(F)c2)C(=O)O1',0.977456],
    'new_64':['CCCCC(NC(=O)Nc1cccc(C(F)(F)F)c1)C(=O)O',0.922584],
    '444':['O=C(O)c1ccc(CCCN2C(=O)CCC2C=CCN2CCC2)s1',0.839427],
    '86':['O=C(O)c1ccc(CCCN2C(=O)CCC2C=CC(O)CCc2ccco2)o1',0.881569],
    '311':['O=C(O)c1ccc(CCCN2C(=O)CCC2C=CC(O)C2CCO2)s1',0.560391],
    '182':['O=C1CCC(C=CC(O)c2cccc(-c3cccc(CO)c3)c2)N1',0.97451],
    '200':['O=C(O)c1ccc(CCCN2C(=O)CCC2C=CCN2CC(F)C2)s1',0.851953],
    '268':['CCNC(=O)C(C)NC(=O)C1CC(=O)N(CCc2ccc(OC)cc2)C1',0.271118],
    '349':['COC(=O)CC1CC(COCc2cccc(-c3ccccc3)c2)NC1=O',0.832895],
    '156':['O=C(CSC1CCNC1=O)NCCOCC(F)(F)F',0.965616]
}

In [22]:
for key, value in smiles_dict.items():
    # print(value)
    smiles = value[0]
    # print(smiles)
    m=Chem.MolFromSmiles(smiles)
    qed = QED.qed(m)
    qed_ = QED.properties(m)
    LogP = qed_.ALOGP
    SAs = sascorer.calculateScore(m)

    value.append(qed)
    value.append(SAs)
    value.append(LogP)

In [29]:
smiles_dict['182']

['O=C1CCC(C=CC(O)c2cccc(-c3cccc(CO)c3)c2)N1',
 0.97451,
 0.7411106298429352,
 3.3550655780834644,
 2.7141]